In [ ]:
#Importamos todas las librerias necesarias y comprobamos si podremos usar la GPU
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
from pandas import *
from PIL import Image as im
import matplotlib.image
print("GPU available: ", tf.config.list_physical_devices('GPU'))
print(tf.__version__)

In [ ]:
DataTest = read_csv("Smiles_Test.csv")
DataTrain = read_csv("Smiles_Train.csv")
SmilesTest = DataTest['SMILES_Test'].tolist()
LogPStringTest = DataTest['LogP_Test'].tolist()
SmilesTrain = DataTrain['SMILES'].tolist()
LogPStringTrain = DataTrain['logP'].tolist()
LogPTrain = np.array(LogPStringTrain)
LogPTest = np.array(LogPStringTest)

In [ ]:
a = 0
MoleculaAsciiTest = []
for x in range(len(SmilesTest)):
    b = 0
    s = SmilesTest[x]
    MolLen = len(SmilesTest[x])
    Arreglo1 = []
    for y in range(529):
        if (b < MolLen):
            Arreglo1.append(ord(s[b]))
        else:
            Arreglo1.append(0)
        b += 1
    MoleculaAsciiTest.append(Arreglo1)
    a += 1

In [ ]:
a = 0
MoleculaAsciiTrain = []
for x in range(len(SmilesTrain)):
    b = 0
    s = SmilesTrain[x]
    MolLen = len(SmilesTrain[x])
    Arreglo1 = []
    for y in range(529):
        if (b < MolLen):
            Arreglo1.append(ord(s[b]))
        else:
            Arreglo1.append(0)
        b += 1
    MoleculaAsciiTrain.append(Arreglo1)
    a += 1

In [ ]:
SmilesTrainImgR = []
for y in range(len(SmilesTrain)):
    SmilesTrainImg = []
    SmilesTrainImg = MoleculaAsciiTrain[y]
    SmilesTrainImg = np.array(SmilesTrainImg)
    SmilesTrainImg = np.reshape(SmilesTrainImg, (23,23))
    SmilesTrainImgR.append(SmilesTrainImg)
    #matplotlib.image.imsave('SmilesTrainImages/Train'+i+'.png',SmilesTrainImg)

In [ ]:
SmilesTestImgR = []
for y in range(len(SmilesTest)):
    SmilesTestImg = []
    SmilesTestImg = MoleculaAsciiTest[y]
    SmilesTestImg = np.array(SmilesTestImg)
    SmilesTestImg = np.reshape(SmilesTestImg, (23,23))
    SmilesTestImgR.append(SmilesTestImg)
    #matplotlib.image.imsave('SmilesTestImages/Test'+i+'.png',SmilesTestImg)

In [ ]:
MoleculaAsciiTrain = np.asarray(MoleculaAsciiTrain)
MoleculaAsciiTest = np.asarray(MoleculaAsciiTest)
SmilesTestImgR = np.asarray(SmilesTestImgR)
SmilesTrainImgR = np.asarray(SmilesTrainImgR)
LogPTrain = np.asarray(LogPTrain)
LogPTest = np.asarray(LogPTest)
MoleculeTensorTrain = tf.convert_to_tensor(MoleculaAsciiTrain)
MoleculeTensorTest = tf.convert_to_tensor(MoleculaAsciiTest)
SmilesTrainImgR = tf.convert_to_tensor(SmilesTrainImgR)
SmilesTestImgR = tf.convert_to_tensor(SmilesTestImgR)

In [ ]:
LogPTrain = LogPTrain.reshape(-1,1)
LogPTest = LogPTest.reshape(-1,1)

In [ ]:
print(SmilesTrainImgR.shape)
print(SmilesTestImgR.shape)
print(LogPTrain.shape)
print(LogPTest.shape)

In [ ]:
normalizador = tf.keras.layers.Normalization(axis=-1)
normalizador.adapt(SmilesTrainImgR)

In [ ]:
modelo = tf.keras.Sequential([
    #tf.keras.layers.Flatten(input_shape=[784]),
    tf.keras.layers.Conv2D(32, (3,3),input_shape=(23,23,1), activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(64, (3,3), activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(units=128, activation=tf.nn.relu),
    tf.keras.layers.Dense(units=128, activation=tf.nn.relu),
    tf.keras.layers.Dense(units=1)
])

In [ ]:
modelo.compile(
    optimizer='adam',
    loss='mean_squared_error',
    #loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    #loss = 'binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
print("Iniciando Entrenamiento...")
modelo.fit(
    SmilesTrainImgR, LogPTrain, epochs = 10,
    verbose = 1
)
print("Entrenamiento finailzado exitosamente")

In [ ]:
test_lost, test_accuracy = modelo.evaluate(SmilesTestImgR)
print("Resultado en las pruebas: ", test_accuracy)

In [ ]:
print("Hagamos una prediccion")
resultado = modelo.predict([SmilesTrainImgR])
print("El resultado es " + str(resultado))

In [ ]:
resultado = np.asarray(resultado)

In [ ]:
archivo = open("Resultado2.dat", "a")
i = 0
resultado1 = []
resultado2 = []
for x in range(100):
    resultado1 = str(resultado[i])
    resultado2 = str(LogPTrain[i])
    resultado1 = resultado1.replace('[', '')
    resultado1 = resultado1.replace(']','')
    resultado2 = resultado2.replace('[', '')
    resultado2 = resultado2.replace(']','')
    archivo.write(str(resultado1)+ ";" + str(resultado2) + "\n")
    i += 1

archivo.close()